<a href="https://colab.research.google.com/github/maryhedy/C551-Tutorials/blob/master/Copy_of_good_accuracy_submission_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mounting the google drive to read the dataset

from google.colab import drive
import pickle 

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2


from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

drive.mount('/content/drive' , force_remount=True)

DATA_PATH = "/content/drive/MyDrive/COMP551-assignment3/data/"

with open(DATA_PATH+'images_l.pkl' ,'rb') as f: images_l=pickle.load(f)
with open(DATA_PATH+'images_test.pkl' ,'rb') as f: images_test=pickle.load(f)
with open(DATA_PATH+'images_ul.pkl' ,'rb') as f: images_ul=pickle.load(f)
with open(DATA_PATH+'labels_l.pkl' ,'rb') as f: labels_l=pickle.load(f)

Mounted at /content/drive


In [ ]:
images_l = images_l/255

numbers = labels_l[:,0:10]
letters = labels_l[:, 10:]


print(images_l.shape)

rgb_batch = np.repeat(images_l[..., np.newaxis], 3, -1)
print(rgb_batch.shape)


(30000, 56, 56)
(30000, 56, 56, 3)


In [ ]:
import matplotlib.pyplot as plt



rgb_batch = np.array([cv2.resize(img, dsize=(64,64), interpolation=cv2.INTER_CUBIC) for img in rgb_batch])



In [ ]:
rgb_batch_val = rgb_batch[28000:]
rgb_batch_train = rgb_batch[0:28000]
numbers_val = numbers[28000:]
numbers_train = numbers[0:28000]

print(rgb_batch_val.shape)
print(numbers_val.shape)
print(rgb_batch_train.shape)
print(numbers_train.shape)

(2000, 64, 64, 3)
(2000, 10)
(28000, 64, 64, 3)
(28000, 10)


In [ ]:
import tensorflow as tf
base_model = tf.keras.applications.resnet50.ResNet50(weights = None, include_top=False, input_shape=(64,64,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(10, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(rgb_batch_train, numbers_train, epochs = 100, batch_size = 64, validation_data=(rgb_batch_val, numbers_val))


Epoch 1/100
422/422 [==============================] - 106s 209ms/step - loss: 3.1759 - accuracy: 0.1217 - val_loss: 25.0776 - val_accuracy: 0.1007
Epoch 2/100
422/422 [==============================] - 83s 198ms/step - loss: 2.6574 - accuracy: 0.1413 - val_loss: 2.3142 - val_accuracy: 0.1467
Epoch 3/100
422/422 [==============================] - 83s 197ms/step - loss: 2.4336 - accuracy: 0.1978 - val_loss: 2.0012 - val_accuracy: 0.2620
Epoch 4/100
422/422 [==============================] - 83s 197ms/step - loss: 2.1063 - accuracy: 0.3202 - val_loss: 10.2644 - val_accuracy: 0.0973
Epoch 5/100
422/422 [==============================] - 83s 196ms/step - loss: 1.7950 - accuracy: 0.4168 - val_loss: 3.7066 - val_accuracy: 0.1923
Epoch 6/100
422/422 [==============================] - 84s 198ms/step - loss: 1.0488 - accuracy: 0.6553 - val_loss: 1.9869 - val_accuracy: 0.5030
Epoch 7/100
422/422 [==============================] - 83s 196ms/step - loss: 0.6392 - accuracy: 0.8055 - val_loss: 5045.

In [ ]:
model.save('./saved_model/number_model')

INFO:tensorflow:Assets written to: ./saved_model/number_model/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
images_test = images_test/255
rgb_batch_test = np.repeat(images_test[..., np.newaxis], 3, -1)
rgb_batch_test = np.array([cv2.resize(img, dsize=(64,64), interpolation=cv2.INTER_CUBIC) for img in rgb_batch_test])

In [ ]:
predictions = model.predict(rgb_batch_test)

In [ ]:
print(predictions.shape)
print(predictions[0])

(15000, 10)
[7.2755404e-02 9.9856954e-04 5.1110696e-06 1.1946436e-04 2.0235123e-02
 1.9489165e-04 2.0223705e-04 9.0393937e-01 1.6552264e-04 1.3842536e-03]


In [ ]:

letters_val = letters[28000:]
letters_train = letters[0:28000]


In [ ]:
base_model_l = tf.keras.applications.resnet50.ResNet50(weights = None, include_top=False, input_shape=(64,64,3))
x_l = base_model_l.output
x_l = GlobalAveragePooling2D()(x_l)
x_l = Dropout(0.7)(x_l)
predictions_l = Dense(26, activation= 'softmax')(x_l)
model_l = Model(inputs = base_model_l.input, outputs = predictions_l)

model_l.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
model_l.fit(rgb_batch_train, letters_train, epochs = 100, batch_size = 64, validation_data=(rgb_batch_val, letters_val))

Epoch 1/100
438/438 [==============================] - 106s 202ms/step - loss: 4.3057 - accuracy: 0.0425 - val_loss: 9.3801 - val_accuracy: 0.0400
Epoch 2/100
438/438 [==============================] - 85s 194ms/step - loss: 3.6421 - accuracy: 0.0481 - val_loss: 4.1615 - val_accuracy: 0.0525
Epoch 3/100
438/438 [==============================] - 85s 195ms/step - loss: 3.5866 - accuracy: 0.0530 - val_loss: 65.2685 - val_accuracy: 0.0425
Epoch 4/100
438/438 [==============================] - 85s 194ms/step - loss: 3.4917 - accuracy: 0.0662 - val_loss: 6.6950 - val_accuracy: 0.0540
Epoch 5/100
438/438 [==============================] - 85s 194ms/step - loss: 3.4760 - accuracy: 0.0718 - val_loss: 1498.0984 - val_accuracy: 0.0420
Epoch 6/100
438/438 [==============================] - 85s 194ms/step - loss: 3.3578 - accuracy: 0.0886 - val_loss: 3.9149 - val_accuracy: 0.0680
Epoch 7/100
438/438 [==============================] - 85s 194ms/step - loss: 3.2693 - accuracy: 0.1113 - val_loss: 3.3

In [ ]:
model_l.save('./saved_model/letter_model')

In [ ]:
predictions_l = model_l.predict(rgb_batch_test)

In [ ]:
arr = np.zeros_like(predictions)
arr[np.arange(len(predictions)), predictions.argmax(1)] = 1 

arr_l = np.zeros_like(predictions_l)
arr_l[np.arange(len(predictions_l)), predictions_l.argmax(1)] = 1 

In [ ]:
solution = np.hstack((arr, arr_l))

In [ ]:
#just for kaggle submission
import pandas

final_list = []
for i, id in enumerate(solution.astype('int')):
  final_list.append([i, ''.join(str(i) for i in id)])
  


df = pandas.DataFrame(final_list)
df.columns = ['# Id','Category']

df.to_csv('./submission.csv', index=False)